In [9]:
import numpy as np
import pandas as pd

import pandas_profiling

In [18]:
movies_df = pd.read_csv(r"ml-1m\movies.dat", delimiter="::", engine="python",
                        names=["id", "name", "genre"], index_col=0, header=None)

In [14]:
ratings_df = pd.read_csv(r"ml-1m\ratings.dat", delimiter="::", engine="python")

In [15]:
users_df = pd.read_csv(r"ml-1m\users.dat", delimiter="::", engine="python")

### EDA

#### Movies

In [19]:
movies_df.head()

,name,genre
id,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy


In [12]:
pandas_profiling.ProfileReport(movies_df)

Number of variables,3
Number of observations,3882
Total Missing (%),0.0%
Total size in memory,91.1 KiB
Average record size in memory,24.0 B
Numeric,1
Categorical,1
Date,0
Text (Unique),1
Rejected,0
Distinct count,3882
